# Quick Notes on Hypothesis Testing

### In general...
* If you have $n \le 15$ samples, use a t-test
* If you have $n \gt 35$ samples, use a z-test
* In between, if variance is known then use normal, if variance is unknown use t distribution
* t-tests and z-tests involve the same calculations, however, they vary in the lookup tables used
<br>

### If you are testing a sample mean to an actual number:
<br>1. $H_0: \mu = 0.5$, $H_a: \mu \ne 0.5$
$$t = \frac{\bar{x} - \mu}{\frac{s}{\sqrt{n}}} = \frac{\bar{x} - 0.5}{\frac{s}{\sqrt{n}}}$$

### If you are testing between two sample populations and you want to do the following tests:
<br>2. $H_0: \mu_1 = \mu_2$, $H_a: \mu_1 \ne \mu_2$ (Two tailed)
<br>3. $H_0: \mu_1 \ge \mu_2$, i.e. $\mu_1 - \mu_2 \ge 0$, $H_a: \mu_1 \lt \mu_2$ (One tailed)
<br>

If the variances are **unequal**, i.e. $\sigma_1^2 \ne \sigma_2^2$, $\color{red}{\text{if t-test}, df \approx n_1 + n_2 -2}$.
$$t_d (or\space z_d) = \frac{\bar{X_1}-\bar{X_2}}{\sqrt{\frac{S_1^2}{n_1}+\frac{S_2^2}{n_2}}}$$
<br>
If variances are **equal** then, $\color{red}{\text{if t-test}, df \approx n_1 + n_2 -2}$.
<br>$$t_d (or\space z_d) = \frac{\bar{x_a}-\bar{x_b}}{\sqrt{S_p^2[\frac{1}{n_a}+\frac{1}{n_b}]}}, \quad S_p^2 = \frac{(n_a - 1)S_a^2 + (n_b - 1)S_b^2}{n_a + n_b - 2} $$
<br><br>
**Note:** You have to conduct an F-test to decide whether population variances are equal (unless explicitly told otherwise)
<br>
### To test variance equality in two sample populations:
<br>4. $H_0: \sigma_a^2 = \sigma_b^2, H_a: \sigma_a^2 \ne \sigma_b^2$ 
<br>$F_{test}=\frac{S_a^2}{S_b^2}$ (need largest F value so put largest variance in the numerator)

### How to interpret a t-value, z-value and p-value

First remember that t-tests require that you know your degrees of freedom, $df$. A z-test does not. 
* If the absolute value of your t-test or z-test is below the critical value indicated by your t-test table or z-test table, then you fail to reject the null hypothesis
* A p-value is very useful because you don't have to do any lookups. A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so you reject the null hypothesis.

### Resources
* [f-test table](http://www.itl.nist.gov/div898/handbook/eda/section3/eda3673.htm)
* [student t test](https://en.wikipedia.org/wiki/Student%27s_t-test)
  * [student t table](https://en.wikipedia.org/wiki/Student%27s_t-distribution#Table_of_selected_values)
* [z-test table](http://www.z-table.com/)
* You could use the scipy.stats.ttest_rel() function here but I want to go through the explicit calculations


## Example 1: test that portfolio A returns are equal to the risk-free annualized return of 3.5%
  * *Hypothesis* 
  <br>$H_0: \bar{x_a} = \mu_{r-free} = 3.5$%, $H_a: \bar{x_a} \ne 3.5$% 
  * $t_{test} = \frac{\bar{x_a}-\mu_{r-free}}{\frac{s_a}{\sqrt{n_a}}} = \frac{\bar{x_a}-.035}{\frac{s_a}{\sqrt{n_a}}}$
<br>

Here we do a t-test since we only have df = 10 - 1 = 9 degrees of freedom. The corresponding critical value is 2.262. Since |-0.168| < 2.262 then we fail to reject the null. In short the returns of portfolio A are not different from 3.5%. 

In [1]:
# Setup the data
import pandas as pd
import numpy as np
data = pd.DataFrame({'A': [123.5, 121.3, 106.5, 102.8, 118.9, 129.6, 137.9, 142.9, 153.7, 162.9, 168.3],
                    'B': [108.6, 101.4, 93.8, 101.9, 112, 119.6, 128.7, 139.5, 145.8, 152.8, 162.5],
                   'year':[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]}).set_index('year')
# Calculate the log returns
returns = np.log(data).diff().dropna()
n = len(returns)
# Calculate mean, var and standard dev of returns
mu_a = returns.mean(axis=0)['A']
var_a = returns.var(axis=0)['A']
s_a = np.sqrt(var_a)
print("Pfol A: mean return = {0:.4f}, variance return = {1:.4f}, std. dev. return = {2:.4f}".format(mu_a, var_a, s_a))
mu_b = returns.mean(axis=0)['B']
var_b = returns.var(axis=0)['B']
s_b = np.sqrt(var_b)
print("Pfol B: mean return = {0:.4f}, variance return = {1:.4f}, std. dev. return = {2:.4f}".format(mu_b, var_b, s_b))

# Do t-test
print("student t statistic = {0:.3f}".format((mu_a - .035)/(s_a/np.sqrt(n))))

Pfol A: mean return = 0.0310, variance return = 0.0058, std. dev. return = 0.0763
Pfol B: mean return = 0.0403, variance return = 0.0038, std. dev. return = 0.0619
student t statistic = -0.168


## Example 2: test that returns for portfolio A are not different from portfolio B returns

First you need to do an F-test of the variances (item 4 in the notes). That result is that the variances of returns between portfolio A and B are not statistically different. Refer to example 4 for that part. Note that *df* = 10+10-2 = 18 so that we need to do a t-test.
<br>
**Hypothesis:**
<br>$H_0: \mu_a = \mu_b$ 
<br>$H_a: \mu_a \ne \mu_b$
$$t_{test} = \frac{\bar{x_a}-\bar{x_b}}{\sqrt{S_p^2[\frac{1}{n_a}+\frac{1}{n_b}]}}, \quad S_p^2 = \frac{(n_a - 1)S_a^2 + (n_b - 1)S_b^2}{n_a + n_b - 2} $$
<br>T-test 95% confidence critical value for df=10+10-2=18 is 2.101
<br>t-test result = -0.301
<br>**Conclusion:** 0.301 < 2.101(critical value) so we fail to reject the null. So portolio A returns are no differnt from portoflio B returns.

In [2]:
var_p = ((n-1)*var_a + (n-1)*var_b)/(n + n - 2)
t = (mu_a - mu_b)/np.sqrt(var_p*(1/n + 1/n))
print("t-statistic = {0:.3f}".format(t))

t-statistic = -0.301


# Example 3: test that portfolio A returns are larger than those of portfolio B

We need a one tailed t-test. And we know that the variance of returns between portfolio A and B are equal.
<br>$H_0: \mu_{a} >= \mu_b$ or $H_0: \mu_{a} - \mu_b >= 0$ 
<br>$H_a: \mu_{a} < \mu_b$ 
<br>$t_{test} = \frac{\bar{x_a}-\bar{x_b}}{\sqrt{S_p^2[\frac{1}{n_a}+\frac{1}{n_b}]}}, \quad S_p^2 = \frac{(n_a - 1)S_a^2 + (n_b - 1)S_b^2}{n_a + n_b - 2} $
<br>
<br>T-test 95% confidence critical value for df=10+10-2=18 is 1.734 (1-sided)
<br>t-test result = -0.301
<br>**Conclusion:** |-0.301| < 1.725(critical value) so we fail to reject the null. Therefore portfolio A does have an equal or higher performance than portfolio B.

In [3]:
var_p = ((n-1)*var_a + (n-1)*var_b)/(n + n - 2)
print("t_test = {0:.3f}".format((mu_a - mu_b)/np.sqrt(var_p*(1/n + 1/n))))

t_test = -0.301


# Example 4: test that portfolio A and B return variances are not different

<br>$H_0: \sigma_a^2 = \sigma_b^2$ 
<br>$H_a: \sigma_a^2 \ne \sigma_b^2$ 
<br>$F_{test}=\frac{S_a^2}{S_b^2}$
<br>Because $F_{.05}(ν_a,ν_b)$ = 2.978 and $F_{test}$ = 1.521 we fail to reject the null hypothesis that the variance of returns in portfolio A is the same as the variance of returns in portfolio B.
<br>

In [4]:
print("F-test, var(a)/var(b) = {0:.3f}".format(var_a/var_b))

F-test, var(a)/var(b) = 1.521
